# Scratchwork

In [2]:
from finite_algebras import *

In [3]:
psr3 = generate_powerset_ring(3)

In [4]:
psr3

Ring(
PSRing3,
Autogenerated ring on powerset of 3 elements w/ symmetric diff. (+) & intersection (*),
['{}', '{0}', '{1}', '{2}', '{0, 1}', '{0, 2}', '{1, 2}', '{0, 1, 2}'],
[[0, 1, 2, 3, 4, 5, 6, 7], [1, 0, 4, 5, 2, 3, 7, 6], [2, 4, 0, 6, 1, 7, 3, 5], [3, 5, 6, 0, 7, 1, 2, 4], [4, 2, 1, 7, 0, 6, 5, 3], [5, 3, 7, 1, 6, 0, 4, 2], [6, 7, 3, 2, 5, 4, 0, 1], [7, 6, 5, 4, 3, 2, 1, 0]]
)